## Using the Azure cognitive services APIs

These are just a few code snippetts to get you started with the Azure cognitive services. Use the ones you like and see how you go!



### 1. Azure Storage API
Almost all Cognitive Services API require input data. i.e. image data to detect faces. This section will show how to use the Azure Storage API to store data in your own cloud storage service with Python APIs.

#### Sign-up to the Azure Storage Service
1. Create an Azure Storage account:
    1. Go to the Azure Portal at http://portal.azure.com/ and sign in with your Azure account credentials
    2. Click the [New] icon on the top left of the Portal, then click Data + Storage > Storage Account. 
    Click the [Create] button, then give the storage account a unique name and create a new resource group for it. 
    
    2. When the storage account has been created, the Notifications button will flash a green SUCCESS flag and the storage account's 'blade' is open to show that it belongs to the new resource group you created.
    3. Click the **Access keys** part in the storage account's blade. Take note of the **account name** and **key1**.

2. Using open source, cross-platform ["Storage Explorer"](http://storageexplorer.com/) tool and the above **account name** and **key1**, create a container under your storage service.

3. You can find the sample files IN THE APPENDIX of this repository. Upload these or similar own files into your Azure storage account, so they cn be used in the python sample codes below.

These activities use the [Microsoft Azure Storage SDK for Python](https://github.com/Azure/azure-storage-python) to access your Azure Storage Service.



In [1]:
#your credentials from above

azure_storage_account_name = 'enterhere'
azure_storage_account_key = 'enterhere'  # dont need key... we will access public blob... 

## IMPORTANT
This line installs the AZURE STORAGE ACCOUNT LIBRARIES required to access Azure Blob Storage Service. 


In [2]:
!pip install azure-storage==0.32.0

    100% |################################| 163kB 3.8MB/s ta 0:00:01
  Found existing installation: azure-storage 0.34.3
    Uninstalling azure-storage-0.34.3:
      Successfully uninstalled azure-storage-0.34.3


In [3]:
from azure.storage.blob import BlockBlobService

# create blob service object to access the files in the storage
# You can access your account_name and account_key values at [Azure Management Portal](https://portal.azure.com)
blob_service = BlockBlobService(azure_storage_account_name, azure_storage_account_key)

List sample data files in a specific container (folder), so we can select one of them to work on it

In [ ]:
# select container (folder) name where the files resides
container_name = 'sampledata'

# list files in the selected folder
generator = blob_service.list_blobs(container_name)

blob_prefix = 'https://{0}.blob.core.windows.net/{1}/{2}'

print("List of files in the container:")
for blob in generator:
    print(blob_prefix.format(blob_service.account_name, container_name, blob.name))

Read one of the image file from the blob storage as byte array. If you want update the below blob_name variable with one of the file available in the above file list

In [ ]:
blob_name = 'emotion1.jpeg'
blob = blob_service.get_blob_to_bytes(container_name, blob_name)

Using PIL and matplotlib libraries, display the image file content to be processed.

In [ ]:
%matplotlib inline

from PIL import Image
import matplotlib.pyplot as plt
import io

image_file_in_mem = io.BytesIO(blob)
img_bytes = Image.open(image_file_in_mem)
plt.imshow(img_bytes)

### 2. SEARCH - Bing Image Search API
You can use Bing Image Search API to keyword based image search, get insights about an image, find trending images.  
Ref: https://msdn.microsoft.com/en-us/library/dn760791.aspx  
     https://www.microsoft.com/cognitive-services/en-us/bing-image-search-api

Call Bing Image Search API to get images with human faces (in later sections, will be used for face detection etc.) Change search parameters as you wish.

In [ ]:
import requests     # used to make rest calls
import json # parse response result
import urllib

url_search_api = 'https://api.cognitive.microsoft.com/bing/v5.0/images/search?' # service address 
api_key = 'c8042821455b4dc095cceaa0cd0dc67e'          # Azure Cognitive API Key, replace with your own key

headers = {'Ocp-Apim-Subscription-Key':api_key}

params = urllib.parse.urlencode({
    'q': 'facial emotion',
    'count': '5',
    'offset': '0',
    'mkt': 'en-us',
    'safeSearch': 'Moderate',
    #'imageContent' : 'Portrait', 
    'imageType' : 'Photo' # AnimatedGif, Clipart, Line, Photo, Shopping
})

url = url_search_api + params

api_response = requests.get(url, headers=headers)

Print out the call response (which is in Json format) in pretty form.

In [ ]:
import json
res_json = json.loads(api_response.content.decode('utf-8'))

print(json.dumps(res_json, indent=2, sort_keys=True))

Display search resulting images

In [ ]:
from IPython.display import Image as ipImage, display
from urllib.parse import urlparse, parse_qs

for j in res_json['value']:
    c_url = j['contentUrl']
    pr = urlparse(c_url)
    ru = parse_qs(pr.query)
    img_url = ru['r'][0]
    img = ipImage(url=img_url, width=150, height=200)
    
    display(img)
    print(img_url)

### 3. VISION - Face API
You can use Face API to Detect, Find Similar, Group, Identify, Verify faces.  
Ref: https://www.microsoft.com/cognitive-services/en-us/face-api

Call Cognitive Services Face API to detect facial features in the image

In [ ]:
# load image file to process
blob_name = 'face4.jpeg'
blob = blob_service.get_blob_to_bytes(container_name, blob_name)
image_file_in_mem = io.BytesIO(blob)
img_bytes = Image.open(image_file_in_mem)

In [ ]:
import requests
import urllib

url_face_api = 'https://api.projectoxford.ai/face/v1.0/detect' # service address 
api_key = 'd469dbc2ba424afd86c0ba0716a62ca8'          # Azure Cognitive API Key, replace with your own key

headers = {'Content-Type': 'application/octet-stream', 'Ocp-Apim-Subscription-Key':api_key}

params = urllib.parse.urlencode({
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'true',
    'returnFaceAttributes': 'age,gender,smile,facialHair,headPose,glasses',
})

query_string = '?{0}'.format(params)

url = url_face_api + query_string

api_response = requests.post(url, headers=headers, data=blob)

Print out the call response (which is in Json format) in pretty form.

In [ ]:
import json
res_json = json.loads(api_response.content.decode('utf-8'))

print(json.dumps(res_json, indent=2, sort_keys=True))

Get detected face coordiates (bounding squares) from call response and draw red square borders for females, blue for males.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches

plt.figure(figsize=(10, 10))
ax = plt.gca()

# Draw face rectangles
for i in res_json:
    fr = i['faceRectangle'] # get faceRectangle node per detected face in the image

    pc = 'red' # patch color
    if i['faceAttributes']['gender'] == 'male':
        pc = 'blue'

    ax.add_patch(
        patches.Rectangle(
            (fr['left'], fr['top']), fr['width'], fr['height'],
            fill=False, linewidth=4, color=pc)
    )
    
    ax.text(fr['left'], fr['top']+fr['height'], 
            'age:'+str(i['faceAttributes']['age']), 
            fontsize=14, weight='bold', color='red', bbox=dict(facecolor='white', alpha=0.8))
    

ps = 5       # patch size
pc = '#00FF00'   # patch color

# Draw eye, nose, mouth
for i in res_json:
    fl = i['faceLandmarks']

    # left eye
    ax.add_patch(patches.Circle((fl['pupilLeft']['x'], fl['pupilLeft']['y']), ps, color=pc))

    # right eye
    ax.add_patch(patches.Circle((fl['pupilRight']['x'], fl['pupilRight']['y']), ps, color=pc))

    # mouth
    ax.add_patch(patches.Circle((fl['mouthLeft']['x'], fl['mouthLeft']['y']), ps, color=pc))
    ax.add_patch(patches.Circle((fl['mouthRight']['x'], fl['mouthRight']['y']), ps, color=pc))

    # nose
    ax.add_patch(patches.Circle((fl['noseTip']['x'], fl['noseTip']['y']), ps, color=pc))

plt.imshow(img_bytes)

### 4. VISION - Emotion API
You can use Emotion API to Detect, emotion recognition in image, in video.  
Ref: https://www.microsoft.com/cognitive-services/en-us/emotion-api

Below sample shows how to call Cognitive Services Emotion API to detect facial emotion in the image

In [ ]:
# load image file to process
blob_name = 'emotion1.jpeg'
blob = blob_service.get_blob_to_bytes(container_name, blob_name)
image_file_in_mem = io.BytesIO(blob)
img_bytes = Image.open(image_file_in_mem)

In [ ]:
import requests
import urllib

url_emotion_api = 'https://api.projectoxford.ai/emotion/v1.0/recognize' # service address 
api_key = '8eae1ba129df43a289015fb45554e074'          # Azure Cognitive API Key, replace with your own key

headers = {'Content-Type': 'application/octet-stream', 'Ocp-Apim-Subscription-Key':api_key}

api_response = requests.post(url_emotion_api, headers=headers, data=blob)

Print out the call response (which is in Json format) in pretty form.

In [ ]:
import json
res_json = json.loads(api_response.content.decode('utf-8'))

print(json.dumps(res_json, indent=2, sort_keys=True))

Get detected face coordiates (bounding squares) from call response and draw borders.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches

plt.figure(figsize=(12, 12))
ax = plt.gca()

# Draw face rectangles
for i in res_json:
    pc = '#00FF00' # patch color
    
    fr = i['faceRectangle'] # get faceRectangle node per detected face in the image
    ax.add_patch(
        patches.Rectangle(
            (fr['left'], fr['top']), fr['width'], fr['height'],
            fill=False, linewidth=4, color=pc)
    )
    
    s = i['scores']
    
    frm_txt = '{0: <10}: {1:>.6f}\n{2: <10}: {3:>.6f}\n{4: <10}: {5:>.6f}\n{6: <10}: {7:>.6f}\n\
{8: <10}: {9:>.6f}\n{10: <10}: {11:>.6f}\n{12: <10}: {13:>.6f}\n{14: <10}: {15:>.6f}'
    
    lbls =     ['anger', s['anger'],\
                'contempt', s['contempt'],\
                'disgust', s['disgust'],\
                'fear', s['fear'],\
                'happiness', s['happiness'],\
                'neutral', s['neutral'],\
                'sadness', s['sadness'],\
                'surprise', s['surprise']]
    
    ax.text(fr['left'], fr['top'], frm_txt.format(*lbls), 
            fontsize=14, weight='bold', color='white', family='monospace',
            bbox=dict(facecolor='green', alpha=0.6))    

plt.imshow(img_bytes)

### 5. VISION - Computer Vision API - Analyze
You can use Computer Vision API to analyze visual content in different ways based on inputs and user choices.  
Ref: https://www.microsoft.com/cognitive-services/en-us/Computer-Vision-API/documentation 

Below sample shows how to call Cognitive Services Computer Vision API to detect image features.

In [ ]:
from IPython.display import Image as ipImage, display
img_url = 'https://portalstoragewuprod.azureedge.net/vision/Analysis/7-1.jpg'
#img_url = 'http://i.ebayimg.com/00/s/NTUxWDg3MQ==/z/IawAAOSw7NNT6NZY/$_32.JPG'
    
img = ipImage(url=img_url, width=250, height=250)
display(img)

In [ ]:
import requests
import urllib

url_vision_api = 'https://api.projectoxford.ai/vision/v1.0/analyze' # service address 
api_key = '3ae10be9a45442f5910434e1af6c52dd'          # Azure Cognitive API Key, replace with your own key

headers = {'Content-Type': 'application/json', 'Ocp-Apim-Subscription-Key':api_key}

params = urllib.parse.urlencode({
    'visualFeatures': 'Categories,Tags,Description,Faces,ImageType,Color,Adult',
})

query_string = '?{0}'.format(params)

url = url_vision_api + query_string

body = '{\'url\':\'' + img_url + '\'}'

api_response = requests.post(url, headers=headers, data=body)

Print out the call response (which is in Json format) in pretty form.

In [ ]:
import json
res_json = json.loads(api_response.content.decode('utf-8'))

print(json.dumps(res_json, indent=2, sort_keys=True))

### 6. VISION - Computer Vision API - OCR
You can use Computer Vision API to read text in images.  
Ref: https://www.microsoft.com/cognitive-services/en-us/Computer-Vision-API/documentation
Below sample shows how to call Cognitive Services Computer Vision API to detect text features.

In [ ]:
# load image file to process
blob_name = 'ocr1.jpeg'
blob = blob_service.get_blob_to_bytes(container_name, blob_name)
image_file_in_mem = io.BytesIO(blob)
img_bytes = Image.open(image_file_in_mem)

# Show the original image that we will process
img_url = 'https://mkmsstrg4ml.blob.core.windows.net/sampledata/ocr1.jpeg'
img = ipImage(url=img_url, width=250, height=250)
display(img)

In [ ]:
import requests
import urllib

url_vision_api = 'https://api.projectoxford.ai/vision/v1.0/ocr' # service address 
api_key = '3ae10be9a45442f5910434e1af6c52dd'          # Azure Cognitive API Key, replace with your own key

headers = {'Content-Type': 'application/octet-stream', 'Ocp-Apim-Subscription-Key':api_key}

params = urllib.parse.urlencode({
    'language': 'unk',
    'detectOrientation': 'true',
})

query_string = '?{0}'.format(params)

url = url_vision_api + query_string

api_response = requests.post(url, headers=headers, data=blob)

Print out the call response (which is in Json format) in pretty form.

In [ ]:
import json
res_json = json.loads(api_response.content.decode('utf-8'))

print(json.dumps(res_json, indent=2, sort_keys=True))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches

plt.figure(figsize=(12, 12))
ax = plt.gca()

text_angle = 0
try:
    text_angle = res_json['textAngle']
except:
    pass

# Draw OCR rectangles
for i in res_json['regions']:
    pl = i['lines']
    for k in pl:
        words = k['words']
        for l in words:
            bb = l['boundingBox']
            txt = l['text']
            
            bb = list(map(int, bb.split(',')))
            
            ax.add_patch(
                patches.Rectangle(
                    (bb[0], bb[1]), bb[2], bb[3], angle=text_angle,
                    fill=False, linewidth=4, color='#00FF00')
            )
            
            ax.text(bb[0], bb[1], txt, 
                    fontsize=14, weight='bold', color='red', bbox=dict(facecolor='white', alpha=0.8))
            
plt.imshow(img_bytes)

### 7. SPEECH - BING Speech API - TTS

Ref: https://www.microsoft.com/cognitive-services/en-us/speech-api/documentation/overview 

The Speech Recognition API provides the ability to convert text to spoken audio by sending text to Microsoftâs servers in the cloud.

In [ ]:
import requests

# Get access token to use the speech services
url_token_api = 'https://api.cognitive.microsoft.com/sts/v1.0/issueToken' # service address 
api_key = '39f61de6d28e4667bbfda88dd37b12ce'          # Azure Cognitive API Key, replace with your own key

headers = {'Content-Length': '0', 'Ocp-Apim-Subscription-Key':api_key}

api_response = requests.post(url_token_api, headers=headers)

access_token = str(api_response.content.decode('utf-8'))


text = '<speak version=\'1.0\' xml:lang=\'en-US\'>\
            <voice xml:lang=\'en-US\' xml:gender=\'Female\' \
                        name=\'Microsoft Server Speech Text to Speech Voice (en-US, ZiraRUS)\'>\
                The Text-To-Speech API enables you to build smart apps that can speak. You can test it now, \
                simply choose your target language, add your sentences then click on the play button to \
                see how speech synthesis works. \
            </voice>\
        </speak>'


# Call Speech to text service
url_tts_api = 'https://speech.platform.bing.com/synthesize' # service address 

headers = {'Authorization': 'Bearer {0}'.format(access_token), \
           'Content-Length': len(text), \
           'Content-type': 'text/plain; charset=utf-8',\
           'X-Microsoft-OutputFormat': 'riff-16khz-16bit-mono-pcm'}

api_response = requests.post(url_tts_api, headers=headers, data=text)

Play the audio content returned by the service call above.

In [ ]:
from IPython.display import Audio, display

audio_bytes = Audio(data=api_response.content)
display(audio_bytes)

### 8. SPEECH - BING Speech API

Ref: https://www.microsoft.com/cognitive-services/en-us/speech-api/documentation/overview 

The Speech Recognition API provides the ability to convert spoken audio to text by sending audio to Microsoftâs servers in the cloud. Developers have a choice of using the REST API or the Client Library (for real-time streaming).

In [4]:
from IPython.display import Audio, display

# load speech file to process
blob_name = 'speech3.wav'
blob = blob_service.get_blob_to_bytes(container_name, blob_name)

wav_bytes = Audio(data=blob)
display(wav_bytes)

NameError: name 'container_name' is not defined

In [5]:
import requests
import urllib
import uuid

# Get access token to use the speech services
url_token_api = 'https://api.cognitive.microsoft.com/sts/v1.0/issueToken' # service address 
api_key = '39f61de6d28e4667bbfda88dd37b12ce'          # Azure Cognitive API Key, replace with your own key

headers = {'Content-Length': '0', 'Ocp-Apim-Subscription-Key':api_key}

api_response = requests.post(url_token_api, headers=headers)

access_token = str(api_response.content.decode('utf-8'))


# Call Speech to text service
url_stt_api = 'https://speech.platform.bing.com/recognize' # service address 

headers = {'Authorization': 'Bearer {0}'.format(access_token), \
           'Content-Length': len(blob), \
           'Content-type': 'audio/wav', \
           'codec': 'audio/pcm', \
           'samplerate': '16000'}

params = urllib.parse.urlencode({
    'scenarios': 'ulm',
    'appid': 'D4D52672-91D7-4C74-8AD8-42B1D98141A5', # dont change, it is fixed by design
    'locale': 'en-US', # speech in english
    'device.os': 'PC',
    'version': '3.0',
    'format': 'json', # return value in json
    'instanceid': str(uuid.uuid1()), # any guid
    'requestid': str(uuid.uuid1()),
})

api_response = requests.post(url_stt_api, headers=headers, params=params, data=blob)

NameError: name 'blob' is not defined

In [ ]:
import json
res_json = json.loads(api_response.content.decode('utf-8'))

print(json.dumps(res_json, indent=2, sort_keys=True))

### 9. LANGUAGE - BING Text Analytics API - Sentiment, Key Phrases, Languages
Extracts information from your text.
ref: https://azure.microsoft.com/en-us/documentation/articles/cognitive-services-text-analytics-quick-start/

In [ ]:
import requests

# Get access token to use the speech services
url_sentiment_api = 'https://westus.api.cognitive.microsoft.com/text/analytics/v2.0/sentiment' # service address 
url_keyphrases_api = 'https://westus.api.cognitive.microsoft.com/text/analytics/v2.0/keyPhrases' # service address 
url_languages_api = 'https://westus.api.cognitive.microsoft.com/text/analytics/v2.0/languages' # service address 
api_key = 'c97786c34cc24041aedfa2a1c64d9aee'          # Azure Cognitive API Key, replace with your own key

headers = {'Content-Length': '0', 'Ocp-Apim-Subscription-Key':api_key}

text = '{                                           \
            "documents": [                          \
                {                                   \
                    "language": "en",               \
                    "id": "1",                      \
                    "text": "remember where we were eight years ago we had the worst financial crisis \
                    the great recession the worst since the nineteen thirties"        \
                },                                  \
            ]                                       \
        }'                                          \


headers = {'Ocp-Apim-Subscription-Key':api_key, \
           'Content-type': 'application/json',\
           'Accept': 'application/json'}

api_response_sentiment = requests.post(url_sentiment_api, headers=headers, data=text)
api_response_keyphrases = requests.post(url_keyphrases_api, headers=headers, data=text)
api_response_languages = requests.post(url_languages_api, headers=headers, data=text)

In [ ]:
import json
res_json_sentiment = json.loads(api_response_sentiment.content.decode('utf-8'))
res_json_keyphrases = json.loads(api_response_keyphrases.content.decode('utf-8'))
res_json_languages = json.loads(api_response_languages.content.decode('utf-8'))

print('// Sentiment analysis result:')
print(json.dumps(res_json_sentiment, indent=2, sort_keys=True))
print('\n\n// Key phrase analysis result:')
print(json.dumps(res_json_keyphrases, indent=2, sort_keys=True))
print('\n\n//Language analysis result:')
print(json.dumps(res_json_languages, indent=2, sort_keys=True))

### 10: Microsoft Academic Knowledge API

As a first step, sign up for access to the API and key your access keys:
    
https://azure.microsoft.com/en-gb/try/cognitive-services/?api=academic-knowledge-api